# Tensor

Tensor 是深度学习框架中极为基础的概念，也是 PyTroch、TensorFlow 中最重要的知识点之一，它是一种数据的存储和处理结构。

几种数据类型：

- 标量，也称 Scalar，是一个只有大小，没有方向的量，比如 1.8、e、10 等。
- 向量，也称 Vector，是一个有大小也有方向的量，比如 (1,2,3,4) 等。
- 矩阵，也称 Matrix，是多个向量合并在一起得到的量，比如 [(1,2,3),(4,5,6)] 等。
- 张量，也称 Tensor

从标量、向量和矩阵的关系来看，它们就是不同“**维度**”的 Tensor ？

更准确地使用 Rank（秩）来表示这种“维度”，比如标量，就是 Rank 为 0 阶的 Tensor；向量就是 Rank 为 1 阶的 Tensor；矩阵就是 Rank 为 2 阶的 Tensor；还有 Rank 大于 2 的 Tensor。

In [1]:
import torch

import numpy as np

## 数据类型

一些常用类型

<img src="https://static001.geekbang.org/resource/image/e6/08/e6af6a3b2172ee08db8c564146ae2108.jpg?wh=1680x933" width="60%" />

## 创建

**直接创建**

```python
torch.tensor(data, dtype=None, device=None,requires_grad=False)
```

- data，是要传入模型的数据。PyTorch 支持通过 list、 tuple、numpy array、scalar 等多种类型进行数据传入，并转换为 tensor；
- dtype，它声明了需要返回的 Tensor 的类型；
- device，这个参数指定了数据要返回到的设备；
- requires_grad，用于说明当前量是否需要在计算中保留对应的梯度信息。

  > requires_grad 的设置要灵活处理：
  >- 如果是训练过程就要设置为 true，目的是方便求导、更新参数。而到了验证或者测试过程；
  >- 如果目的是检查当前模型的泛化能力，就要设置成 Fasle，避免这个参数根据 loss 自动更新。

**从 Numpy 创建**

```python
torch.from_numpy(ndarry)
```

**创建特殊形式的 Tensor**

```python
torch.zeros(*size, dtype=None...)
torch.ones(size, dtype=None...)
```

**创建单位矩阵 Tensor**（单位矩阵是指主对角线上的元素都为 1 的矩阵）

```python
torch.eye(size, dtype=None...)
```

**创建随机矩阵 Tensor** 有以下几种较常使用的方式：

```python
torch.rand(size)
torch.randn(size)
torch.normal(mean, std, size)
torch.randint(low, high, size)
```

- torch.rand 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数据在 0~1 区间均匀分布。
- torch.randn 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数的取值满足均值为 0、方差为 1 的标准正态分布。
- torch.normal 用于生成数据类型为浮点型且维度指定的随机 Tensor，可以指定均值和标准差。
- torch.randint 用于生成随机整数的 Tensor，其内部填充的是在[low,high) 均匀生成的随机整数。


## Tensor 的转换

**Int 与 Tensor 的转换**

通过 torch.tensor 将一个数字（或者标量）转换为 Tensor，又通过 item() 函数，将 Tensor 转换为数字（标量），item() 函数的作用就是将 Tensor 转换为一个 python number。

In [2]:
a = torch.tensor(1)
b = a.item()
print(a)
print(b)

tensor(1)
1


**list 与 Tensor 的转换**

使用 torch.tensor 就可以将其转换为 Tensor 了。而还原回来的过程要多一步，需要先将 Tensor 转为 NumPy 结构，之后再使用 tolist() 函数得到 list。

In [3]:
a = [1, 2, 3]
b = torch.tensor(a)
c = b.numpy().tolist()

print(b)
print(c)

tensor([1, 2, 3])
[1, 2, 3]


**NumPy 与 Tensor 的转换**

使用 torch.tensor

In [4]:
np_data = np.arange(6).reshape((2, 3))

torch_data = torch.tensor(np_data)
# torch_data = torch.from_numpy(np_data)

tensor2array = torch_data.numpy()

print(
    "\nnumpy array:\n", np_data,
    "\ntorch tensor:\n", torch_data,
    "\ntensor to array:\n", tensor2array,
)


numpy array:
 [[0 1 2]
 [3 4 5]] 
torch tensor:
 tensor([[0, 1, 2],
        [3, 4, 5]]) 
tensor to array:
 [[0 1 2]
 [3 4 5]]


In [5]:
# torch.tensor(np_data)
torch_data

tensor([[0, 1, 2],
        [3, 4, 5]])

**CPU 与 GPU 的 Tensor 之间的转换**

```
CPU->GPU: data.cuda()
GPU->CPU: data.cpu()
```

## Tensor 的常用操作

**获取形状**

使用 shape 或 size 来获取。两者的不同之处在于，shape 是 Tensor对象 的一个属性，而 size() 则是一个 Tensor对象 拥有的方法

In [6]:
a = torch.zeros(2, 3, 5)
print(a.shape)
print(a.size())

torch.Size([2, 3, 5])
torch.Size([2, 3, 5])


**获取元素数量**

知道了 Tensor 的形状，就很容易得到 Tensor 所包含的元素的数量了。具体的计算方法就是直接将所有维度的大小相乘，比如上面的 Tensor a 所含有的元素的个数为 2 * 3 * 5=30 个。

In [7]:
a.numel()

30

**矩阵转秩 (维度转换）**

在 PyTorch 中有两个函数： permute() 和 transpose() 可以用来实现矩阵的转秩，或者说交换不同维度的数据。
比如在调整卷积层的尺寸、修改 channel 的顺序、变换全连接层的大小的时候，就要用到它们。

- 函数 permute ，可以对任意高维矩阵进行转置，但只有 tensor.permute() 这个调用方式。
- 函数 transpose，不同于 permute，它每次只能转换两个维度，或者说交换两个维度的数据。

<img src="https://static001.geekbang.org/resource/image/02/84/025985c8635f3896d45d15e1ea381c84.jpg" width="35%" />

In [8]:
x = torch.rand(2, 3, 5)
print(x.shape)

# 2 表示原来第二个维度现在放在了第零个维度；
# 1 表示原来第一个维度仍旧在第一个维度；
# 0 表示原来第 0 个维度放在了现在的第 2 个维度。
# 最后，形状就变成 [5,3,2]
x = x.permute(2, 1, 0)
print(x.shape)


torch.Size([2, 3, 5])
torch.Size([5, 3, 2])


In [9]:
x = torch.rand(2, 3, 4)
print(x.shape)
x = x.transpose(1, 0)
print(x.shape)

torch.Size([2, 3, 4])
torch.Size([3, 2, 4])


In [10]:
x = torch.rand(2, 3, 5)
print(x.shape)

x = x.transpose(0, 2)
print(x.shape)

torch.Size([2, 3, 5])
torch.Size([5, 3, 2])


经过了 transpose 或者 permute 处理之后的数据，变得不再连续了。内存虽然没有变化，但是的数据第 0 和第 2 维的数据发生了交换，现在的第 0 维是原来的第 2 维，所以 Tensor 都会变得不再连续。

不连续，有啥影响？接着看。

**形状变换**

在 PyTorch 中有两种常用的改变形状的函数，分别是 view 和 reshape。


In [11]:
x = torch.randn(4, 4)
print(x.shape)
x = x.view(2, 8)
print(x.shape)
x

torch.Size([4, 4])
torch.Size([2, 8])


tensor([[-0.6063, -1.3673, -0.1813, -0.1494,  0.4834, -1.7225,  0.0171,  1.8201],
        [ 0.3441,  1.5952, -0.0528, -0.0723,  0.6460, -0.0282,  2.3335, -0.9216]])

In [12]:
x = x.permute(1, 0)
print(x.shape)

# x.view(4, 4)

# RuntimeError: view size is not compatible with input tensor's size and stride
# (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

torch.Size([8, 2])


抛出错误是由于经过 permute 后，将第 0 和第 1 维度的数据进行了变换，Tensor 变得不再连续了，而 view 无法处理不连续的 Tensor 。

而 reshape 可以很好地工作，因为 reshape 相当于进行了两步操作，先把 Tensor 在内存中捋顺了，然后再进行 view 操作。

In [13]:
x = x.reshape(4, 4)
x

tensor([[-0.6063,  0.3441, -1.3673,  1.5952],
        [-0.1813, -0.0528, -0.1494, -0.0723],
        [ 0.4834,  0.6460, -1.7225, -0.0282],
        [ 0.0171,  2.3335,  1.8201, -0.9216]])

**增减维度**

有时候我们需要对 Tensor 增加或者删除某些维度，比如删除或者增加图片的几个通道。PyTorch 提供了 squeeze() 和 unsqueeze() 函数解决这个问题。

- squeeze()：如果 dim 指定的维度的值为 1，则将该维度删除，若指定的维度值不为 1，则返回原来的 Tensor。
- unsqueeze()：这个函数主要是对数据维度进行扩充。给指定位置加上维数为 1 的维度。

In [14]:
x = torch.rand(2, 1, 3)
print(x.shape)

y = x.squeeze(1)
print(y.shape)

# 这里失败了：因为 y 此刻的第 1 维度的大小为 3，suqeeze 不能删除。
z = y.squeeze(1)
print(z.shape)

torch.Size([2, 1, 3])
torch.Size([2, 3])
torch.Size([2, 3])


In [15]:
x = torch.rand(2,1,3)
y = x.unsqueeze(2)
y.shape

torch.Size([2, 1, 1, 3])

## 小结

把 NumPy 和 Tensor 做对比，它们之间有很多共通的内容，两者都是数据的表示形式，都可以看作是科学计算的通用工具。但是 NumPy 和 Tensor 的用途是不一样的，NumPy 不能用于 GPU 加速，Tensor 则可以。


## Tensor 的连接

在项目开发中，深度学习某一层神经元的数据可能有多个不同的来源，那么就需要将数据进行组合，这个组合的操作，称之为连接。

**cat**

cat 连接操作函数，全称 concatnate。

```python
torch.cat(tensors, dim = 0, out = None)
```
- 第一个参数是 tensors，它很好理解，就是若干个准备进行拼接的 Tensor。
- 第二个参数是 dim，是指拼接的维度（秩）。

下面以二维 Tensor 来体验一下。

In [16]:
A = torch.ones(3, 3)
B = 2 * torch.ones(3, 3)

print(A)
print(B)

# 拼接 dim=0
# 按照“行”的方向拼接
C = torch.cat([A, B], 0)
print(C)

# dim=1
# 按照“列”的方向拼接的
D = torch.cat([A, B], 1)
print(D)


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])
tensor([[1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.]])


无论 Tensor 是三维甚至更高维，cat 都会按照 dim 的数值维度方向链接两个 Tensor。

cat 是将多个 Tensor 在已有的维度上进行连接，如果想增加新的维度进行连接，就需要 stack 函数登场了。

**stack**

假设有两个二维矩阵 Tensor，把它们“堆叠”放在一起，构成一个三维的 Tensor 。这相当于原来的维度（秩）是 2，现在变成了 3，变成了一个立体的结构，增加了一个维度。

```python
torch.stack(inputs, dim=0)
```

inputs 表示需要拼接的 Tensor，dim 表示新建立维度的方向。


In [17]:
A = torch.arange(0, 4)
print(A)
B = torch.arange(5, 9)
print(B)

# 在 dim=0，也就是“行”的方向上新建一个维度
C = torch.stack((A, B), 0)
print(C)

# 在 dim=1，也就是“列”的方向上新建维度
D = torch.stack((A, B), 1)
print(D)

tensor([0, 1, 2, 3])
tensor([5, 6, 7, 8])
tensor([[0, 1, 2, 3],
        [5, 6, 7, 8]])
tensor([[0, 5],
        [1, 6],
        [2, 7],
        [3, 8]])


## Tensor 的切分

切分就是连接的逆过程，切分的操作主要分为三种类型：chunk、split、unbind。

**chunk**

chunk 的作用就是将 Tensor 按照声明的 dim，进行尽可能平均的划分。

```python
torch.chunk(input, chunks, dim=0)
```

- input，它表示要做 chunk 操作的 Tensor。
- chunks，它代表将要被划分的块的数量，而不是每组的数量。请注意，chunks 必须是整型。
- dim，就是按照哪个维度来进行 chunk。

>chunk 参数不能够整除，chunk 函数是先做除法，然后再**向上取整**得到每组的数量。

In [18]:
A = torch.arange(1, 11)
B = torch.chunk(A, 2, 0)
print(A)
print(B)

# math.ceil(10 / 3)
C = torch.chunk(A, 3, 0)
print(C)
for t in C:
    print(t.shape)

# 18 / 4
A1 = torch.arange(1, 18)
C1 = torch.chunk(A1, 4, 0)
print(C1)
for t in C1:
    print(t.shape)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
(tensor([1, 2, 3, 4, 5]), tensor([ 6,  7,  8,  9, 10]))
(tensor([1, 2, 3, 4]), tensor([5, 6, 7, 8]), tensor([ 9, 10]))
torch.Size([4])
torch.Size([4])
torch.Size([2])
(tensor([1, 2, 3, 4, 5]), tensor([ 6,  7,  8,  9, 10]), tensor([11, 12, 13, 14, 15]), tensor([16, 17]))
torch.Size([5])
torch.Size([5])
torch.Size([5])
torch.Size([2])


In [19]:
A=torch.arange(1,17).reshape((4,4))
print(A)

B = torch.chunk(A, 2, 0)
print(B)
for t in B:
    print(t.shape)

C = torch.chunk(A, 2, 1)
C

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
(tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]]), tensor([[ 9, 10, 11, 12],
        [13, 14, 15, 16]]))
torch.Size([2, 4])
torch.Size([2, 4])


(tensor([[ 1,  2],
         [ 5,  6],
         [ 9, 10],
         [13, 14]]),
 tensor([[ 3,  4],
         [ 7,  8],
         [11, 12],
         [15, 16]]))

**split**

按照“每份按照确定的大小”来进行切分

```python
torch.split(tensor, split_size_or_sections, dim=0)
```

- tensor，也就是待切分的 Tensor。
- split_size_or_sections ，
  - 当它为整数时，表示将 tensor 按照每块大小为这个整数的数值来切割；
  - 当这个参数为列表时，则表示将此 tensor 切成和列表中元素一样大小的块。
- dim，它定义了要按哪个维度切分。


In [20]:
A = torch.arange(1, 21).reshape(5, 4)
print(A)

# 按照每组2”列“的大小进行切分，得到了两个 5x2 大小的 Tensor
C = torch.split(A, 2, 1)
print(C)

# 如果 split_size_or_sections 不能整除对应方向的大小
# 得到一个 3x4 和一个 2x4 大小的 Tensor
D = torch.split(A, 3, 0)
print(D)

# split_size_or_sections 是列表时的情况
# 将此 tensor 切成和列表中元素大小一样的大小的块
E = torch.split(A, (2, 3), 0)
print(E)

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20]])
(tensor([[ 1,  2],
        [ 5,  6],
        [ 9, 10],
        [13, 14],
        [17, 18]]), tensor([[ 3,  4],
        [ 7,  8],
        [11, 12],
        [15, 16],
        [19, 20]]))
(tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]]), tensor([[13, 14, 15, 16],
        [17, 18, 19, 20]]))
(tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]]), tensor([[ 9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20]]))


**unbind**

unbind 是一种降维切分的方式，相当于删除一个维度之后的结果。

```python
torch.unbind(input, dim=0)
```

- input 表示待处理的 Tensor，
- dim 表示切片的方向。

In [21]:
A = torch.arange(0, 16).view(4, 4)
print(A)
B = torch.unbind(A, 0)
C = torch.unbind(A, 1)

print(B)
print(C)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
(tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([ 8,  9, 10, 11]), tensor([12, 13, 14, 15]))
(tensor([ 0,  4,  8, 12]), tensor([ 1,  5,  9, 13]), tensor([ 2,  6, 10, 14]), tensor([ 3,  7, 11, 15]))


## 索引操作

上面的操作都是对数据整体进行切分，并获得全部结果。有时候，我们只需要其中的一部分，要怎么做呢？直接告诉 Tensor 想要哪些部分，这种方法被称为索引操作。

最常用的两个操作就是 index_select 和 masked_select 。

**index_select**

基于给定的索引来进行数据提取。

```python
torch.index_select(tensor, dim, index)
```

In [22]:
A = torch.arange(0, 16).view(4, 4)
print(A)

# 第 0 维选择第 1（行）和 3（行）的数据
B = torch.index_select(A, 0, torch.tensor([1, 3]))
print(B)

# 选择第 0（列）和 3（列）的数据
C = torch.index_select(A, 1, torch.tensor([0, 3]))
print(C)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[ 4,  5,  6,  7],
        [12, 13, 14, 15]])
tensor([[ 0,  3],
        [ 4,  7],
        [ 8, 11],
        [12, 15]])


**masked_select**

通过一些判断条件来进行数据提取。

```python
torch.masked_select(input, mask, out=None)
```

- input 表示待处理的 Tensor。
- mask 代表掩码张量，也就是满足条件的特征掩码。需要注意：mask 须跟 input 张量的最高Rank/维的元素数相等，但形状或维度不需要相同。
  比如，Rank = 2，那么就是列数目相同。

In [23]:
A = torch.rand(5)
print(A)

B = A > 0.3
print(B)

# A 中 “满足 B 里面元素值为 True 的” 对应位置的数据
C = torch.masked_select(A, B)
print(C)

# 上面可以简化为
D = torch.masked_select(A, A > 0.3)
print(D)

tensor([0.5941, 0.9153, 0.9745, 0.9504, 0.2550])
tensor([ True,  True,  True,  True, False])
tensor([0.5941, 0.9153, 0.9745, 0.9504])
tensor([0.5941, 0.9153, 0.9745, 0.9504])


In [24]:
A = torch.arange(1, 21).reshape(4, 5)
print(A)
print(A.shape)

# mask 的最高维元素数目 == A 的高维元素数目 5
mask = torch.tensor([True, False, False, True, False])
print(mask)
print(mask.shape)

torch.masked_select(A, mask)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20]])
torch.Size([4, 5])
tensor([ True, False, False,  True, False])
torch.Size([5])


tensor([ 1,  4,  6,  9, 11, 14, 16, 19])

## 小结

Tensor 中的主要函数跟用法

<img src="https://static001.geekbang.org/resource/image/d1/ba/d195706087f784c8e1e1c7c7b25a22ba.jpg?wh=3020x2455" width="60%" />

## 思考题

有这样一个 tensor，要提取出其中第一行的第一个，第二行的第一、第二个，第三行的最后一个，该怎么做呢？

```python
A=torch.tensor([[4,5,7], [3,9,8],[2,3,4]])
```

In [25]:
A = torch.tensor([[4, 5, 7], [3, 9, 8], [2, 3, 4]])
torch.masked_select(A, torch.tensor([[True, False, False],[True, True, False], [False, False, True]]))

tensor([4, 3, 9, 4])